In [ ]:
import os
import numpy as np
import pandas as pd
import keras as k
import matplotlib as plt
import seaborn as sns; sns.set()
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import models, layers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras import metrics
import tensorflow as tf
from pandas_ml import ConfusionMatrix
import matplotlib.pyplot as plt

In [ ]:
! pip install pandas_ml

In [ ]:
c=2
np.random.seed(1)
if c==1:
    df = pd.read_csv("data_ml.txt", sep=';' ,error_bad_lines=False)
    df = df[df["iyear"]>=2000].drop(columns="Unnamed: 0")
    df = df[df.gname != 'Unknown']
    df = df[["gname","imonth","iday","country","region","latitude","longitude","attacktype1","targtype1","targsubtype1","natlty1"]]
    a = df['gname'].value_counts().nlargest(10)
    #df[df['gname']=='Boko Haram'].append(df[df['gname']=='Islamic State of Iraq and the Levant (ISIL)']).append(df[df['gname']=='Taliban'])
    df_filtered = pd.DataFrame()
    for i in a.index:
        b = df[df.gname == i]
        df_filtered = df_filtered.append(b)
    
    df = df_filtered
    df = df.fillna(-99)
    lb_make = LabelEncoder()
    df["gname_num"] = lb_make.fit_transform(df["gname"])
    
    X = df.values[:, 1:11]
    Y = df.values[:, 11]
    Y = to_categorical(Y)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
else:
    df = pd.read_csv("data_ml.txt", sep=";" ,error_bad_lines=False)
    df = df[df["iyear"]>=2000].drop(columns="Unnamed: 0")
    df = df[df.gname != 'Unknown']
    df = df[["gname","imonth","iday","country","region","latitude","longitude","attacktype1","targtype1","targsubtype1","natlty1"]]
    df = df.fillna(-99)
    lb_make = LabelEncoder()
    df["gname_num"] = lb_make.fit_transform(df["gname"])
    
    X = df.values[:, 1:11]
    Y = df.values[:, 11]
    Y = to_categorical(Y)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
np.random.seed(1)    
model1 = Sequential()
model1.add(Dense(10, activation='relu', input_shape=(X.shape[1],), use_bias=True, bias_initializer='zeros'))
model1.add(Dropout(0.1))
model1.add(Dense(100, activation='relu', use_bias=True, bias_initializer='zeros'))
model1.add(Dropout(0.1))
model1.add(Dense(250, activation='relu', use_bias=True, bias_initializer='zeros'))
model1.add(Dropout(0.2))
model1.add(Dense(100, activation='relu', use_bias=True, bias_initializer='zeros'))
model1.add(Dropout(0.1))
model1.add(Dense(output_dim=Y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc',"mae","mse"])

model1.fit(x_train, y_train, epochs=10)

In [ ]:
preds = model1.predict(x_test)
#np.argmax(preds, axis=None, out=None)
y_preds = [np.argmax(p, axis=None, out=None) for p in preds]
y_test_labels = [np.argmax(y, axis=None, out=None) for y in y_test] 

In [ ]:
accuracy_score(y_test_labels, y_preds)

In [ ]:
cm = confusion_matrix(y_preds, y_test_labels)
cm.shape

In [ ]:
np.random.seed(1)    
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(X.shape[1],), use_bias=True, bias_initializer='zeros'))
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu', use_bias=True, bias_initializer='zeros'))
model.add(Dropout(0.1))
model.add(Dense(250, activation='relu', use_bias=True, bias_initializer='zeros'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu', use_bias=True, bias_initializer='zeros'))
model.add(Dropout(0.1))
model.add(Dense(output_dim=Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc',"mae","mse"])

model.fit(x_train, y_train, epochs=50)

In [ ]:
preds = model.predict(x_test)
#np.argmax(preds, axis=None, out=None)
y_preds = [np.argmax(p, axis=None, out=None) for p in preds]
y_test_labels = [np.argmax(y, axis=None, out=None) for y in y_test] 

In [ ]:
accuracy_score(y_test_labels, y_preds)

In [ ]:
cm = confusion_matrix(lb_make.inverse_transform(y_preds), lb_make.inverse_transform(y_test_labels))
cm

In [ ]:
#from google.colab import files
cm = ConfusionMatrix(lb_make.inverse_transform(y_preds), lb_make.inverse_transform(y_test_labels))
cm.plot()
#plt.savefig( "cm.png", bbox_inches='tight', dpi=200) 
#files.download("cm.png") 

In [ ]:
london = np.array([6, #imonth
          3, #iday
          603, #country
          8, #region
          51.508297, #lat
          -0.087703, #lon
          2, #attacktype1
          18, #targtype1
          97, #targszbtype1
          216, #nalty1
          #3, #nperps
          #9, #weaptype1
          #9, #weaptype2
          #10, #weaptype3
          #11, #nkill
          #48, #nwound
          #0, #property
          #3, #propextent
          #0 #nhostkid
         ]).reshape(1, 10)

In [ ]:
#create list 5 likeliest organisations + probabilities for london attack
ld = model1.predict(london)
b =[]
a = (-ld).argsort()[:5]
for i in range(0,5):
    b.append([lb_make.inverse_transform([a.item(i)])[0], ld.item(a.item(i))])
    
# Plot the prediction
names = []
values = []
for i in range(0,5):
    names.append(b[i][0])
    values.append(b[i][1])
    
fig, ax = plt.subplots()
#plt.title("London 2017 Borough Market (Top 5)")
sns.barplot(y=names, x=values, palette='inferno')
ax.set_xlabel('Probabilty Score')

plt.savefig("london.png",bbox_inches='tight', dpi=200)
plt.show()

In [ ]:
paris = np.array([11, #imonth
          13, #iday
          69, #country
          8, #region
          48.862883, #lat
           2.370326, #lon
          2, #attacktype1
          14, #targtype1
          81, #targszbtype1
          69, #nalty1
          #9, #nperps
          #5, #weaptype1
          #6, #weaptype2
          #5, #weaptype3
          #137, #nkill
          #413, #nwound
          #0, #property
          #3, #propextent
          #12 #nhostkid
         ]).reshape(1, 10)

In [ ]:
#create list 5 likeliest organisations + probabilities for paris attack 
pr = model1.predict(paris)
b =[]
a = (-pr).argsort()[:5]
for i in range(0,5):
    b.append([lb_make.inverse_transform([a.item(i)])[0], pr.item(a.item(i))])
    
# Plot the prediction
names = []
values = []
for i in range(0,5):
    names.append(b[i][0])
    values.append(b[i][1])
    
fig, ax = plt.subplots()
#plt.title("Paris 2015 Bataclan Theater (Top 5)")
sns.barplot(y=names, x=values, palette='inferno')
ax.set_xlabel('Probabilty Score')

plt.savefig("paris.png",bbox_inches='tight', dpi=200)
plt.show()  